### Select N-words with higher TF

In [26]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *

In [5]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

,text,summary
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


In [6]:
all_tf = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    all_tf.append(tf(l))

In [7]:
all_tf

[{'congenital': 1.0,
  '##ren': 7.0,
  '##al': 7.0,
  'hyper': 2.0,
  '##pl': 2.0,
  '##asia': 1.0,
  '##h': 17.0,
  'refers': 1.0,
  'group': 1.0,
  'auto': 1.0,
  '##som': 1.0,
  'recess': 1.0,
  '##ive': 3.0,
  'disorders': 1.0,
  'caused': 1.0,
  'enzyme': 1.0,
  'deficiency': 2.0,
  'which': 5.0,
  'leads': 1.0,
  'defects': 1.0,
  'bio': 2.0,
  '##sy': 1.0,
  '##nt': 1.0,
  '##hesis': 1.0,
  'ste': 1.0,
  '##roid': 1.0,
  'precursor': 2.0,
  '##s': 5.0,
  '.': 43.0,
  'depending': 1.0,
  'the': 32.0,
  'severity': 1.0,
  'and': 26.0,
  'degree': 1.0,
  'hydro': 3.0,
  '##xy': 2.0,
  '##lase': 1.0,
  'clinical': 2.0,
  'spectrum': 1.0,
  'may': 2.0,
  'vary': 1.0,
  'from': 2.0,
  'mild': 1.0,
  'form': 1.0,
  'non': 3.0,
  'classical': 3.0,
  'classic': 5.0,
  'however': 3.0,
  'variant': 1.0,
  'more': 1.0,
  'common': 2.0,
  'with': 23.0,
  'prevalence': 1.0,
  'rate': 1.0,
  '1000': 1.0,
  'also': 4.0,
  'helps': 1.0,
  'maintaining': 1.0,
  'normal': 2.0,
  'levels': 6.0,
  '

In [24]:
def best_words_format(tfs, n):
    output = "Maximize\nscore: "
    for i, v in enumerate(tfs.values()):
        output += f"+ {v} c{i}"
    
    output += "\n\nSubject To\n"
    output += "length:"
    for i, v in enumerate(tfs.values()):
        output += f" c{i} +"
    output = output[:-1]
    output += f"< {n}"

    output += "\n\nBinary\n"
    for i, v in enumerate(tfs.values()):
        output += f"c{i}\n"
    return output


In [30]:
format = best_words_format(all_tf[0], 4)

with open(os.path.join(get_git_root(), r"myLibraries\ilp_outputs\ilp_in.ilp"), "w") as f:
    f.write(format)
    f.close()

os.system(f'glpsol --tmlim 100 --lp "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_in.ilp")}" -o "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol")}"')

with open(os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol"), "r") as f:
    lines = f.readlines()
    f.close()

sentences_lines = [line for line in lines if re.search(r"c\d", line)]

sorted_lines = sorted(sentences_lines, key=lambda line: int(line.split()[1][1:]))
result = [int(sorted_line.split()[3]) for sorted_line in sorted_lines]

In [33]:
index = [i for i, value in enumerate(result) if value == 1]
index

[28, 30, 32, 91]

In [41]:
ref = []
for i, (k, v) in enumerate(all_tf[0].items()):
    ref.append([i, k, v])
ref.sort(key=lambda x: x[2], reverse=True)
res = [v[0] for v in ref[:4]]
res

[28, 30, 32, 91]

### Select N-sentences maximizing words tfs